In [ ]:
import urllib.request
import pickle, csv, json, os, re
import numpy as np
from asyncore import read

from time import sleep
from tqdm import tqdm

We define the functions used to gather data from the jikan APi https://api.jikan.moe

In [ ]:
def get_num_pages(link):
    r = urllib.request.urlopen(link)
    text = r.read().decode('utf-8')
    js = json.loads(text)
    last_page = js['pagination']['last_visible_page']
    return last_page


def get_all_pages(page):
    pages = []
    max_pages = get_num_pages('https://api.jikan.moe/v4/{}?page={:.2f}&sfw=true'.format(page,0))
    for i in tqdm(range(max_pages)):
        r = urllib.request.urlopen('https://api.jikan.moe/v4/{}?page={:.2f}&sfw=true'.format(page,i))
        text = r.read().decode('utf-8')
        js = json.loads(text)
        keys = js['data']
        for k in keys:
            pages.append(k)
        sleep(0.75) #The API only allows 3 requests per second, so we have to make sure we don't make more than three requests/sec
    return pages

Now, we extract all anime pages, and save them as text files locally, so they can be re-used

In [ ]:
all_animes = get_all_pages('anime')

for anime in all_animes:
    file_name = "{}.txt".format(anime['mal_id'])
    write_file = open("data/animes/"+file_name, "w")
    write_file.write(json.dumps(anime))
    write_file.close()

Similarly, we extract all manga pages, and save them locally

In [ ]:
all_mangas = get_all_pages('manga')

for manga in all_mangas:
    file_name = "{}.txt".format(manga['mal_id'])
    write_file = open("data/mangas/"+file_name, "w")
    write_file.write(json.dumps(manga))
    write_file.close()